In [1]:
import nest_asyncio

nest_asyncio.apply()
# 1. AGENT LEVEL
import os
from agents import Agent, OpenAIChatCompletionsModel, AsyncOpenAI, Runner
import asyncio
try:

    GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
    if not GEMINI_API_KEY:
        raise ValueError("GEMINI ApI key not found")
  
    external_client = AsyncOpenAI(
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
        api_key=GEMINI_API_KEY,
    )
    model = OpenAIChatCompletionsModel(
        model="gemini-2.0-flash", openai_client=external_client
    )
except Exception as e:
    print("Error", e)

In [71]:
from agents import Agent, function_tool, RunHooks
import json
import requests
from datetime import date
from typing import Optional,Any
from pydantic import BaseModel 
import asyncio

class OutputType(BaseModel):
    valid:bool
    resp_natural:str
 
    
def dynamic_comp_validator(context, agent):
    user_query = context.context["user_query"]
    order_data = context.context["order_data"]
    today = date.today().strftime("%Y-%m-%d")

    return (
    f"You are a complaint validation assistant for **ArmanShoe Mart**.\n\n"
    f"Today's date is {today}.\n\n"
    f"Your task is to determine whether a customer's complaint is valid based only on the order details and their complaint description.\n\n"
    
    "**Order Details:**\n"
    f"{json.dumps(order_data, indent=2)}\n\n"

    "**Customer Complaint Description:**\n"
    f"{user_query}\n\n"

    "**Validation Rules:**\n"
    "1. The complaint is invalid if:"
    "   - The order `status` is not 'received'.\n"
    "   → Even if today's date is before the expected delivery date, once status is 'received', treat it as delivered.\n"
    "   → In all other cases, assume the product is not yet delivered.\n\n"

    "2. If the complaint mentions incorrect size, color, or quantity, strictly verify that these claims match the order data.\n"
    "   - If the customer says they ordered size 10 but the order has size 7 or 9 → it's invalid.\n"
    "   - If the customer says they ordered 10 items, but the order data shows they only ordered 2 — not received 2, but ordered 2 — then the complaint is invalid.\n"
    "   - Only declare it valid if **all the issues mentioned** clearly exist in the order data.\n"
    "3. Do not make assumptions, and never be lenient. Follow the facts.\n\n"

    "**Your Response Format:**\n"
    "```json\n"
    "{\n"
    '  "valid": true/false,\n'
    '  "resp_natural": "Your polite response to the customer in clear natural language."\n'
    "}\n"
    "```\n\n"
    "Use friendly and professional language in the `resp_natural` field to explain whether the complaint is valid or not. Make sure it's helpful and concise."
    "**Examples:**\n"
    "- User says they ordered 10 shoes, but system shows 4 → invalid.\n"
    "- User says they received wrong color, but order color matches → invalid.\n"
    "- Complaint before delivery date → invalid.\n"
    "- Everything matches complaint → valid.\n\n"

    "**Important:** Always validate strictly, based on order facts. Never guess. Never assume. Only confirm if complaint matches data exactly."
    "**Important:** Compare against what was actually ordered, not what the user says they received. Do not assume under-delivery unless the data supports it"
)


 
@function_tool
async def customer_complaint_fun(email: str, tracking_id: str, comp_description: str) -> dict:
    """
    Register and return complaint-related details for a user's order.

    Args:
        email (str): The customer's email address.
        tracking_id (str): The order tracking ID the complaint relates to.
        comp_description (str): A description of the issue or complaint.

    Returns:
        dict: Complaint summary, order details, and item-level breakdown.
    """
    
    resp = {
        "orders": [
            {
                "shipping_address": "village lakra sialkot,pakistan",
                "order_id": "ASM91411184",
                "status": "pending",
                "expected_delivery_date": "2025-08-03",
                "created_at": "2025-04-26",
                "items": [
                    {
                        "product_name": "Nike Burrow SE",
                        "price": 45.97,
                        "quantity": 2,
                        "color": "Black/Black",
                        "size": 7,
                        "image": "https://static.nike.com/a/images/t_default/43929bf5-d412-4267-84fa-a544a9bfe004/W+NIKE+BURROW+SE+NA.png",
                    },
                    {
                        "product_name": "Nike Burrow SE",
                        "price": 45.97,
                        "quantity": 2,
                        "color": "Black/Black",
                        "size": 9,
                        "image": "https://static.nike.com/a/images/t_default/43929bf5-d412-4267-84fa-a544a9bfe004/W+NIKE+BURROW+SE+NA.png",
                    },
                ],
            }
        ]
    }
 
    new_context = {"user_query": my_context, "order_data": resp} 
    # print(f"new;{new_context}")
   
    comp_validator = Agent(
        name="complaint_validator",
        instructions=dynamic_comp_validator,model=model,output_type=OutputType
        
    )
    resp = await Runner.run(comp_validator, input="",  context=new_context,)
    # print(f"on tool end final response : {resp.final_output}")
    final_resp =  resp.final_output
    return final_resp
     


CustomerComplaint = Agent(
    name="customer_complaint_agent",
    instructions="""You are a customer support assistant helping users submit complaints related to their orders.\n"
        "Use the customer's email, order tracking ID, and complaint description to register the complaint "
        "and retrieve related order information.\n\n"
        "If any required fields are missing, politely ask the user to provide them.\n"
        "You must call the appropriate tool with complete data.""",
    model=model,
    tools=[customer_complaint_fun],
    tool_use_behavior="stop_on_first_tool",
 
)


async def run_customer_complaint(query):
    global my_context 
    my_context= query
    
    return await Runner.run(CustomerComplaint, query)


 
if __name__ =='__main__':
    query =  "i want to submit my complaint my email is armanashraf015@gmail.com and my tracking id is ASM91411184 my product box are open"
    result = asyncio.run(run_customer_complaint(query))
    print(result.final_output)

valid=False resp_natural="Thank you for reaching out to us. We have reviewed your complaint regarding order ASM91411184. Currently, the order status is 'pending,' and the expected delivery date is 2025-08-03. Since the order has not yet been delivered, we cannot validate complaints about the condition of the delivered product at this time. Please contact us again after you receive your order."


In [53]:
from agents import Agent, function_tool, AgentHooks
import json
import requests
from datetime import date
from pydantic import BaseModel

class OutputType(BaseModel):
    valid:bool
    resp_natural:str

def dynamic_comp_validator(context, agent):
    user_query = context.context["user_query"]
    order_data = context.context["order_data"]
    today = date.today().strftime("%Y-%m-%d")

    return (
    f"You are a complaint validation assistant for **ArmanShoe Mart**.\n\n"
    f"Today's date is {today}.\n\n"
    f"Your task is to determine whether a customer's complaint is valid based only on the order details and their complaint description.\n\n"
    
    "**Order Details:**\n"
    f"{json.dumps(order_data, indent=2)}\n\n"

    "**Customer Complaint Description:**\n"
    f"{user_query}\n\n"

    "**Validation Rules:**\n"
    "1. The complaint is invalid if:"
    "   - The order `status` is not 'received'.\n"
    "   → Even if today's date is before the expected delivery date, once status is 'received', treat it as delivered.\n"
    "   → In all other cases, assume the product is not yet delivered.\n\n"

    "2. If the complaint mentions incorrect size, color, or quantity, strictly verify that these claims match the order data.\n"
    "   - If the customer says they ordered size 10 but the order has size 7 or 9 → it's invalid.\n"
    "   - If the customer says they ordered 10 items, but the order data shows they only ordered 2 — not received 2, but ordered 2 — then the complaint is invalid.\n"
    "   - Only declare it valid if **all the issues mentioned** clearly exist in the order data.\n"
    "3. Do not make assumptions, and never be lenient. Follow the facts.\n\n"

    "**Your Response Format:**\n"
    "```json\n"
    "{\n"
    '  "valid": true/false,\n'
    '  "resp_natural": "Your polite response to the customer in clear natural language."\n'
    "}\n"
    "```\n\n"
    "Use friendly and professional language in the `resp_natural` field to explain whether the complaint is valid or not. Make sure it's helpful and concise."
    "**Examples:**\n"
    "- User says they ordered 10 shoes, but system shows 4 → invalid.\n"
    "- User says they received wrong color, but order color matches → invalid.\n"
    "- Complaint before delivery date → invalid.\n"
    "- Everything matches complaint → valid.\n\n"

    "**Important:** Always validate strictly, based on order facts. Never guess. Never assume. Only confirm if complaint matches data exactly."
    "**Important:** Compare against what was actually ordered, not what the user says they received. Do not assume under-delivery unless the data supports it"
)


class CustomHook(AgentHooks):

    async def on_tool_end(self, context, agent, tool, output):
        try:
            new_context = {"user_query": context.context, "order_data": output} 
            comp_validator = Agent(
                name="complaint_validator",
                instructions=dynamic_comp_validator,model=model,output_type=OutputType
              
            )
            resp = await Runner.run(comp_validator, input="",  context=new_context,)
            final_resp = resp.final_output
            print(f'final_resp:{final_resp}')
            return final_resp

        except Exception as e:
            err = str(e)
            if "API key not valid" in err or "API_KEY_INVALID" in err:
                code, msg = "400", "Invalid API key. Please provide a valid API key."
            elif "RESOURCE_EXHAUSTED" in err or "quota" in err.lower():
                code, msg = (
                    "429",
                    "Quota exceeded. Please upgrade or wait before retrying.",
                )
            elif isinstance(e, requests.exceptions.ConnectionError):
                code, msg = "500", "Network error. Please check your connection."
            elif "model is overloaded" in err.lower() or "503" in err:
                code, msg = (
                    "503",
                    "Gemini model is overloaded. Please try again after a few seconds.",
                )
            else:
                code, msg = "API_ERROR", f"Unexpected API error: {err}"

            raise RuntimeError(f"{code}: {msg}") from e


@function_tool
def customer_complaint_fun(email: str, tracking_id: str, comp_description: str) -> dict:
    """
    Register and return complaint-related details for a user's order.

    Args:
        email (str): The customer's email address.
        tracking_id (str): The order tracking ID the complaint relates to.
        comp_description (str): A description of the issue or complaint.

    Returns:
        dict: Complaint summary, order details, and item-level breakdown.
    """
    print(f"comp_description:{comp_description}")
    resp = {
        "orders": [
            {
                "shipping_address": "village lakra sialkot,pakistan",
                "order_id": "ASM91411184",
                "status": "received",
                "expected_delivery_date": "2025-08-03",
                "created_at": "2025-04-26",
                "items": [
                    {
                        "product_name": "Nike Burrow SE",
                        "price": 45.97,
                        "quantity": 2,
                        "color": "Black/Black",
                        "size": 7,
                        "image": "https://static.nike.com/a/images/t_default/43929bf5-d412-4267-84fa-a544a9bfe004/W+NIKE+BURROW+SE+NA.png",
                    },
                   
                ],
            }
        ]
    }
    return resp


CustomerComplaint = Agent(
    name="customer_complaint_agent",
    instructions="""You are a customer support assistant helping users submit complaints related to their orders.\n"
        "Use the customer's email, order tracking ID, and complaint description to register the complaint "
        "and retrieve related order information.\n\n"
        "If any required fields are missing, politely ask the user to provide them.\n"
        "You must call the appropriate tool with complete data.""",
    model=model,
    tools=[customer_complaint_fun],
    tool_use_behavior="stop_on_first_tool",
    hooks=CustomHook(),
)

async def run_customer_complaint():
    query =  "i want to submit my complaint my email is armanashraf015@gmail.com and my tracking id is ASM91411184 my products are open and broken"
   
 
    return await Runner.run(CustomerComplaint, query, context=query)

import asyncio

async def main():
    result = await run_customer_complaint()
    return result
if __name__ =='__main__':
    result = asyncio.run(main())
    # print(result.final_output)

comp_description:my products are open and broken
final_resp:valid=True resp_natural="Thank you for reaching out to us. We have reviewed your complaint regarding order ASM91411184. The order status is 'received'. You mentioned that the products were open and broken. Since the order has been received, your complaint is valid."


In [49]:
from agents import Agent, function_tool, AgentHooks
import json
import requests
from datetime import date
from pydantic import BaseModel

class OutputType(BaseModel):
    valid:bool
    resp_natural:str

def dynamic_comp_validator(context, agent):
    user_query = context.context["user_query"]
    order_data = context.context["order_data"]
    today = date.today().strftime("%Y-%m-%d")

    return (
    f"You are a complaint validation assistant for **ArmanShoe Mart**.\n\n"
    f"Today's date is {today}.\n\n"
    f"Your task is to determine whether a customer's complaint is valid based only on the order details and their complaint description.\n\n"
    
    "**Order Details:**\n"
    f"{json.dumps(order_data, indent=2)}\n\n"

    "**Customer Complaint Description:**\n"
    f"{user_query}\n\n"

    "**Validation Rules:**\n"
    "1. The complaint is invalid if **any** of the following is true:\n"
    "   - Today's date is **before** the expected delivery date.\n"
    "   - The order `status` is not `'received'` (e.g., it's still `'pending'`).\n"
    "   → In these cases, explain that the item has not yet been delivered, so no complaint can be accepted.\n\n"

    "2. If the complaint mentions incorrect size, color, or quantity, strictly verify that these claims match the order data.\n"
    "   - If the customer says they ordered size 10 but the order has size 7 or 9 → it's invalid.\n"
    "   - If the customer says they ordered 10 items, but the order data shows they only ordered 2 — not received 2, but ordered 2 — then the complaint is invalid.\n"
    "   - Only declare it valid if **all the issues mentioned** clearly exist in the order data.\n"
    "3. Do not make assumptions, and never be lenient. Follow the facts.\n\n"

    "**Your Response Format:**\n"
    "```json\n"
    "{\n"
    '  "valid": true/false,\n'
    '  "resp_natural": "Your polite response to the customer in clear natural language."\n'
    "}\n"
    "```\n\n"
    "Use friendly and professional language in the `resp_natural` field to explain whether the complaint is valid or not. Make sure it's helpful and concise."
    "**Examples:**\n"
    "- User says they ordered 10 shoes, but system shows 4 → invalid.\n"
    "- User says they received wrong color, but order color matches → invalid.\n"
    "- Complaint before delivery date → invalid.\n"
    "- Everything matches complaint → valid.\n\n"

    "**Important:** Always validate strictly, based on order facts. Never guess. Never assume. Only confirm if complaint matches data exactly."
    "**Important:** Compare against what was actually ordered, not what the user says they received. Do not assume under-delivery unless the data supports it"
)


class CustomHook(AgentHooks):

    async def on_tool_end(self, context, agent, tool, output):
        try:
            new_context = {"user_query": context.context, "order_data": output} 
            comp_validator = Agent(
                name="complaint_validator",
                instructions=dynamic_comp_validator,model=model,output_type=OutputType
              
            )
            resp = await Runner.run(comp_validator, input="",  context=new_context,)
            final_resp = resp.final_output
            print(f'final_resp:{final_resp}')
            return final_resp

        except Exception as e:
            err = str(e)
            if "API key not valid" in err or "API_KEY_INVALID" in err:
                code, msg = "400", "Invalid API key. Please provide a valid API key."
            elif "RESOURCE_EXHAUSTED" in err or "quota" in err.lower():
                code, msg = (
                    "429",
                    "Quota exceeded. Please upgrade or wait before retrying.",
                )
            elif isinstance(e, requests.exceptions.ConnectionError):
                code, msg = "500", "Network error. Please check your connection."
            elif "model is overloaded" in err.lower() or "503" in err:
                code, msg = (
                    "503",
                    "Gemini model is overloaded. Please try again after a few seconds.",
                )
            else:
                code, msg = "API_ERROR", f"Unexpected API error: {err}"

            raise RuntimeError(f"{code}: {msg}") from e


@function_tool
def customer_complaint_fun(email: str, tracking_id: str, comp_description: str) -> dict:
    """
    Register and return complaint-related details for a user's order.

    Args:
        email (str): The customer's email address.
        tracking_id (str): The order tracking ID the complaint relates to.
        comp_description (str): A description of the issue or complaint.

    Returns:
        dict: Complaint summary, order details, and item-level breakdown.
    """
    print(f"comp_description:{comp_description}")
    resp = {
        "orders": [
            {
                "shipping_address": "village lakra sialkot,pakistan",
                "order_id": "ASM91411184",
                "status": "pending",
                "expected_delivery_date": "2025-08-03",
                "created_at": "2025-04-26",
                "items": [
                    {
                        "product_name": "Nike Burrow SE",
                        "price": 45.97,
                        "quantity": 2,
                        "color": "Black/Black",
                        "size": 7,
                        "image": "https://static.nike.com/a/images/t_default/43929bf5-d412-4267-84fa-a544a9bfe004/W+NIKE+BURROW+SE+NA.png",
                    },
                   
                ],
            }
        ]
    }
    return resp


CustomerComplaint = Agent(
    name="customer_complaint_agent",
    instructions="""You are a customer support assistant helping users submit complaints related to their orders.\n"
        "Use the customer's email, order tracking ID, and complaint description to register the complaint "
        "and retrieve related order information.\n\n"
        "If any required fields are missing, politely ask the user to provide them.\n"
        "You must call the appropriate tool with complete data.""",
    model=model,
    tools=[customer_complaint_fun],
    tool_use_behavior="stop_on_first_tool",
    hooks=CustomHook(),
)


async def run_customer_complaint():
    query =  "i want to submit my complaint my email is armanashraf015@gmail.com and my tracking id is ASM91411184 we buy ten quantity but recived only 2 quantity"
   
 
    return await Runner.run(CustomerComplaint, query, context=query)

import asyncio

async def main():
    result = await run_customer_complaint()
    return result
if __name__ =='__main__':
    result = asyncio.run(main())
    # print(result.final_output)

comp_description:we buy ten quantity but recived only 2 quantity
final_resp:valid=False resp_natural='Thank you for contacting ArmanShoe Mart. We have reviewed your complaint regarding order ASM91411184. According to our records, the order is currently pending and has an expected delivery date of 2025-08-03. As the expected delivery date has not yet passed, we are unable to process your complaint at this time. Once you have received the order, please contact us again if there are any issues.'


In [48]:
from agents import Agent, function_tool, AgentHooks
import json
import requests
from datetime import date
from pydantic import BaseModel

class OutputType(BaseModel):
    valid:bool
    resp_natural:str

def dynamic_comp_validator(context, agent):
    user_query = context.context["user_query"]
    order_data = context.context["order_data"]
    today = date.today().strftime("%Y-%m-%d")

    return (
    f"You are a complaint validation assistant for **ArmanShoe Mart**.\n\n"
    f"Today's date is {today}.\n\n"
    f"Your task is to determine whether a customer's complaint is valid based only on the order details and their complaint description.\n\n"
    
    "**Order Details:**\n"
    f"{json.dumps(order_data, indent=2)}\n\n"

    "**Customer Complaint Description:**\n"
    f"{user_query}\n\n"

    "**Validation Rules:**\n"
    "1. The complaint is invalid if **any** of the following is true:\n"
    "   - Today's date is **before** the expected delivery date.\n"
    "   - The order `status` is not `'received'` (e.g., it's still `'pending'`).\n"
    "   → In these cases, explain that the item has not yet been delivered, so no complaint can be accepted.\n\n"

    "2. If the complaint mentions incorrect size, color, or quantity, strictly verify that these claims match the order data.\n"
    "   - If the customer says they ordered size 10 but the order has size 7 or 9 → it's invalid.\n"
    "   - If the customer says they ordered 10 items, but the order data shows they only ordered 2 — not received 2, but ordered 2 — then the complaint is invalid.\n"
    "   - Only declare it valid if **all the issues mentioned** clearly exist in the order data.\n"
    "3. Do not make assumptions, and never be lenient. Follow the facts.\n\n"

    "**Your Response Format:**\n"
    "```json\n"
    "{\n"
    '  "valid": true/false,\n'
    '  "resp_natural": "Your polite response to the customer in clear natural language."\n'
    "}\n"
    "```\n\n"
    "Use friendly and professional language in the `resp_natural` field to explain whether the complaint is valid or not. Make sure it's helpful and concise."
    "**Examples:**\n"
    "- User says they ordered 10 shoes, but system shows 4 → invalid.\n"
    "- User says they received wrong color, but order color matches → invalid.\n"
    "- Complaint before delivery date → invalid.\n"
    "- Everything matches complaint → valid.\n\n"

    "**Important:** Always validate strictly, based on order facts. Never guess. Never assume. Only confirm if complaint matches data exactly."
    "**Important:** Compare against what was actually ordered, not what the user says they received. Do not assume under-delivery unless the data supports it"
)


class CustomHook(AgentHooks):

    async def on_tool_end(self, context, agent, tool, output):
        try:
            new_context = {"user_query": context.context, "order_data": output} 
            comp_validator = Agent(
                name="complaint_validator",
                instructions=dynamic_comp_validator,model=model,output_type=OutputType
              
            )
            resp = await Runner.run(comp_validator, input="",  context=new_context,)
            final_resp = resp.final_output
            print(f'final_resp:{final_resp}')
            return final_resp

        except Exception as e:
            err = str(e)
            if "API key not valid" in err or "API_KEY_INVALID" in err:
                code, msg = "400", "Invalid API key. Please provide a valid API key."
            elif "RESOURCE_EXHAUSTED" in err or "quota" in err.lower():
                code, msg = (
                    "429",
                    "Quota exceeded. Please upgrade or wait before retrying.",
                )
            elif isinstance(e, requests.exceptions.ConnectionError):
                code, msg = "500", "Network error. Please check your connection."
            elif "model is overloaded" in err.lower() or "503" in err:
                code, msg = (
                    "503",
                    "Gemini model is overloaded. Please try again after a few seconds.",
                )
            else:
                code, msg = "API_ERROR", f"Unexpected API error: {err}"

            raise RuntimeError(f"{code}: {msg}") from e


@function_tool
def customer_complaint_fun(email: str, tracking_id: str, comp_description: str) -> dict:
    """
    Register and return complaint-related details for a user's order.

    Args:
        email (str): The customer's email address.
        tracking_id (str): The order tracking ID the complaint relates to.
        comp_description (str): A description of the issue or complaint.

    Returns:
        dict: Complaint summary, order details, and item-level breakdown.
    """
    print(f"comp_description:{comp_description}")
    resp = {
        "orders": [
            {
                "shipping_address": "village lakra sialkot,pakistan",
                "order_id": "ASM91411184",
                "status": "pending",
                "expected_delivery_date": "2025-08-03",
                "created_at": "2025-04-26",
                "items": [
                    {
                        "product_name": "Nike Burrow SE",
                        "price": 45.97,
                        "quantity": 2,
                        "color": "Black/Black",
                        "size": 7,
                        "image": "https://static.nike.com/a/images/t_default/43929bf5-d412-4267-84fa-a544a9bfe004/W+NIKE+BURROW+SE+NA.png",
                    },
                   
                ],
            }
        ]
    }
    return resp


CustomerComplaint = Agent(
    name="customer_complaint_agent",
    instructions="""You are a customer support assistant helping users submit complaints related to their orders.\n"
        "Use the customer's email, order tracking ID, and complaint description to register the complaint "
        "and retrieve related order information.\n\n"
        "If any required fields are missing, politely ask the user to provide them.\n"
        "You must call the appropriate tool with complete data.""",
    model=model,
    tools=[customer_complaint_fun],
    tool_use_behavior="stop_on_first_tool",
    hooks=CustomHook(),
)

async def run_customer_complaint():
    query =  "i want to submit my complaint my email is armanashraf015@gmail.com and my tracking id is ASM91411184 "
   
 
    return await Runner.run(CustomerComplaint, query, context=query)

import asyncio

async def main():
    result = await run_customer_complaint()
    return result
if __name__ =='__main__':
    result = asyncio.run(main())
    print(result.final_output)

Please provide a description of your complaint so I can register it.



In [43]:
from agents import Agent, function_tool, AgentHooks
import json
import requests
from datetime import date
from pydantic import BaseModel

class OutputType(BaseModel):
    valid:bool
    reason:str

def dynamic_comp_validator(context, agent):
    user_query = context.context["user_query"]
    order_data = context.context["order_data"]
    today = date.today().strftime("%Y-%m-%d")

    return (
    f"You are a complaint validation assistant for **ArmanShoe Mart**.\n\n"
    f"Today's date is {today}.\n\n"
    f"Your task is to determine whether a customer's complaint is valid based only on the order details and their complaint description.\n\n"
    
    "**Order Details:**\n"
    f"{json.dumps(order_data, indent=2)}\n\n"

    "**Customer Complaint Description:**\n"
    f"{user_query}\n\n"

    "**Validation Rules:**\n"
    "1. The complaint is invalid if **any** of the following is true:\n"
    "   - Today's date is **before** the expected delivery date.\n"
    "   - The order `status` is not `'received'` (e.g., it's still `'pending'`).\n"
    "   → In these cases, explain that the item has not yet been delivered, so no complaint can be accepted.\n\n"

    "2. If the complaint mentions incorrect size, color, or quantity, strictly verify that these claims match the order data.\n"
    "   - If the customer says they ordered size 10 but the order has size 7 or 9 → it's invalid.\n"
    "   - If the customer says they ordered 10 items, but the order data shows they only ordered 2 — not received 2, but ordered 2 — then the complaint is invalid.\n"
    "   - Only declare it valid if **all the issues mentioned** clearly exist in the order data.\n"
    "3. Do not make assumptions, and never be lenient. Follow the facts.\n\n"

    "**Your Response Format:**\n"
    "```json\n"
    "{\n"
    '  "valid": true/false,\n'
    '  "reason": "<short explanation>"\n'
    "}\n"
    "```\n\n"

    "**Examples:**\n"
    "- User says they ordered 10 shoes, but system shows 4 → invalid.\n"
    "- User says they received wrong color, but order color matches → invalid.\n"
    "- Complaint before delivery date → invalid.\n"
    "- Everything matches complaint → valid.\n\n"

    "**Important:** Always validate strictly, based on order facts. Never guess. Never assume. Only confirm if complaint matches data exactly."
    "**Important:** Compare against what was actually ordered, not what the user says they received. Do not assume under-delivery unless the data supports it"
)


class CustomHook(AgentHooks):

    async def on_tool_end(self, context, agent, tool, output):
        try:
            new_context = {"user_query": context.context, "order_data": output} 
            comp_validator = Agent(
                name="complaint_validator",
                instructions=dynamic_comp_validator,model=model,output_type=OutputType
              
            )
            resp = await Runner.run(comp_validator, input="",  context=new_context,)
            final_resp = resp.final_output
            print(f'final_resp:{final_resp}')
            return final_resp

        except Exception as e:
            err = str(e)
            if "API key not valid" in err or "API_KEY_INVALID" in err:
                code, msg = "400", "Invalid API key. Please provide a valid API key."
            elif "RESOURCE_EXHAUSTED" in err or "quota" in err.lower():
                code, msg = (
                    "429",
                    "Quota exceeded. Please upgrade or wait before retrying.",
                )
            elif isinstance(e, requests.exceptions.ConnectionError):
                code, msg = "500", "Network error. Please check your connection."
            elif "model is overloaded" in err.lower() or "503" in err:
                code, msg = (
                    "503",
                    "Gemini model is overloaded. Please try again after a few seconds.",
                )
            else:
                code, msg = "API_ERROR", f"Unexpected API error: {err}"

            raise RuntimeError(f"{code}: {msg}") from e


@function_tool
def customer_complaint_fun(email: str, tracking_id: str, comp_description: str) -> dict:
    """
    Register and return complaint-related details for a user's order.

    Args:
        email (str): The customer's email address.
        tracking_id (str): The order tracking ID the complaint relates to.
        comp_description (str): A description of the issue or complaint.

    Returns:
        dict: Complaint summary, order details, and item-level breakdown.
    """
    print(f"comp_description:{comp_description}")
    resp = {
        "orders": [
            {
                "shipping_address": "village lakra sialkot,pakistan",
                "order_id": "ASM91411184",
                "status": "pending",
                "expected_delivery_date": "2025-08-03",
                "created_at": "2025-04-26",
                "items": [
                    {
                        "product_name": "Nike Burrow SE",
                        "price": 45.97,
                        "quantity": 2,
                        "color": "Black/Black",
                        "size": 7,
                        "image": "https://static.nike.com/a/images/t_default/43929bf5-d412-4267-84fa-a544a9bfe004/W+NIKE+BURROW+SE+NA.png",
                    },
                   
                ],
            }
        ]
    }
    return resp


CustomerComplaint = Agent(
    name="customer_complaint_agent",
    instructions="""You are a customer support assistant helping users submit complaints related to their orders.\n"
        "Use the customer's email, order tracking ID, and complaint description to register the complaint "
        "and retrieve related order information.\n\n"
        "If any required fields are missing, politely ask the user to provide them.\n"
        "You must call the appropriate tool with complete data.""",
    model=model,
    tools=[customer_complaint_fun],
    tool_use_behavior="stop_on_first_tool",
    hooks=CustomHook(),
)


async def run_customer_complaint():
    query =  "i want to submit my complaint my email is armanashraf015@gmail.com and my tracking id is ASM91411184 we orderd black shoes but we recived white shoes "
   
 
    return await Runner.run(CustomerComplaint, query, context=query)

import asyncio

async def main():
    result = await run_customer_complaint()
    return result
if __name__ =='__main__':
    result = asyncio.run(main())
    print(result.final_output)

comp_description:we orderd black shoes but we recived white shoes
final_resp:valid=False reason='The order status is pending, and the expected delivery date is in the future. The complaint cannot be accepted until the order is received.'
{'orders': [{'shipping_address': 'village lakra sialkot,pakistan', 'order_id': 'ASM91411184', 'status': 'pending', 'expected_delivery_date': '2025-08-03', 'created_at': '2025-04-26', 'items': [{'product_name': 'Nike Burrow SE', 'price': 45.97, 'quantity': 2, 'color': 'Black/Black', 'size': 7, 'image': 'https://static.nike.com/a/images/t_default/43929bf5-d412-4267-84fa-a544a9bfe004/W+NIKE+BURROW+SE+NA.png'}]}]}


In [47]:
from agents import Agent, function_tool, AgentHooks
import json
import requests
from datetime import date
from pydantic import BaseModel

class OutputType(BaseModel):
    valid:bool
    resp_natural:str

def dynamic_comp_validator(context, agent):
    user_query = context.context["user_query"]
    order_data = context.context["order_data"]
    today = date.today().strftime("%Y-%m-%d")

    return (
    f"You are a complaint validation assistant for **ArmanShoe Mart**.\n\n"
    f"Today's date is {today}.\n\n"
    f"Your task is to determine whether a customer's complaint is valid based only on the order details and their complaint description.\n\n"
    
    "**Order Details:**\n"
    f"{json.dumps(order_data, indent=2)}\n\n"

    "**Customer Complaint Description:**\n"
    f"{user_query}\n\n"

    "**Validation Rules:**\n"
    "1. The complaint is invalid if **any** of the following is true:\n"
    "   - Today's date is **before** the expected delivery date.\n"
    "   - The order `status` is not `'received'` (e.g., it's still `'pending'`).\n"
    "   → In these cases, explain that the item has not yet been delivered, so no complaint can be accepted.\n\n"

    "2. If the complaint mentions incorrect size, color, or quantity, strictly verify that these claims match the order data.\n"
    "   - If the customer says they ordered size 10 but the order has size 7 or 9 → it's invalid.\n"
    "   - If the customer says they ordered 10 items, but the order data shows they only ordered 2 — not received 2, but ordered 2 — then the complaint is invalid.\n"
    "   - Only declare it valid if **all the issues mentioned** clearly exist in the order data.\n"
    "3. Do not make assumptions, and never be lenient. Follow the facts.\n\n"

    "**Your Response Format:**\n"
    "```json\n"
    "{\n"
    '  "valid": true/false,\n'
    '  "resp_natural": "Your polite response to the customer in clear natural language."\n'
    "}\n"
    "```\n\n"
    "Use friendly and professional language in the `resp_natural` field to explain whether the complaint is valid or not. Make sure it's helpful and concise."
    "**Examples:**\n"
    "- User says they ordered 10 shoes, but system shows 4 → invalid.\n"
    "- User says they received wrong color, but order color matches → invalid.\n"
    "- Complaint before delivery date → invalid.\n"
    "- Everything matches complaint → valid.\n\n"

    "**Important:** Always validate strictly, based on order facts. Never guess. Never assume. Only confirm if complaint matches data exactly."
    "**Important:** Compare against what was actually ordered, not what the user says they received. Do not assume under-delivery unless the data supports it"
)


class CustomHook(AgentHooks):

    async def on_tool_end(self, context, agent, tool, output):
        try:
            new_context = {"user_query": context.context, "order_data": output} 
            comp_validator = Agent(
                name="complaint_validator",
                instructions=dynamic_comp_validator,model=model,output_type=OutputType
              
            )
            resp = await Runner.run(comp_validator, input="",  context=new_context,)
            final_resp = resp.final_output
            print(f'final_resp:{final_resp}')
            return final_resp

        except Exception as e:
            err = str(e)
            if "API key not valid" in err or "API_KEY_INVALID" in err:
                code, msg = "400", "Invalid API key. Please provide a valid API key."
            elif "RESOURCE_EXHAUSTED" in err or "quota" in err.lower():
                code, msg = (
                    "429",
                    "Quota exceeded. Please upgrade or wait before retrying.",
                )
            elif isinstance(e, requests.exceptions.ConnectionError):
                code, msg = "500", "Network error. Please check your connection."
            elif "model is overloaded" in err.lower() or "503" in err:
                code, msg = (
                    "503",
                    "Gemini model is overloaded. Please try again after a few seconds.",
                )
            else:
                code, msg = "API_ERROR", f"Unexpected API error: {err}"

            raise RuntimeError(f"{code}: {msg}") from e


@function_tool
def customer_complaint_fun(email: str, tracking_id: str, comp_description: str) -> dict:
    """
    Register and return complaint-related details for a user's order.

    Args:
        email (str): The customer's email address.
        tracking_id (str): The order tracking ID the complaint relates to.
        comp_description (str): A description of the issue or complaint.

    Returns:
        dict: Complaint summary, order details, and item-level breakdown.
    """
    print(f"comp_description:{comp_description}")
    resp = {
        "orders": [
            {
                "shipping_address": "village lakra sialkot,pakistan",
                "order_id": "ASM91411184",
                "status": "pending",
                "expected_delivery_date": "2025-08-03",
                "created_at": "2025-04-26",
                "items": [
                    {
                        "product_name": "Nike Burrow SE",
                        "price": 45.97,
                        "quantity": 2,
                        "color": "Black/Black",
                        "size": 7,
                        "image": "https://static.nike.com/a/images/t_default/43929bf5-d412-4267-84fa-a544a9bfe004/W+NIKE+BURROW+SE+NA.png",
                    },
                   
                ],
            }
        ]
    }
    return resp


CustomerComplaint = Agent(
    name="customer_complaint_agent",
    instructions="""You are a customer support assistant helping users submit complaints related to their orders.\n"
        "Use the customer's email, order tracking ID, and complaint description to register the complaint "
        "and retrieve related order information.\n\n"
        "If any required fields are missing, politely ask the user to provide them.\n"
        "You must call the appropriate tool with complete data.""",
    model=model,
    tools=[customer_complaint_fun],
    tool_use_behavior="stop_on_first_tool",
    hooks=CustomHook(),
)

async def run_customer_complaint():
    query =  "i want to submit my complaint my email is armanashraf015@gmail.com and my tracking id is ASM91411184 we orderd black shoes but we recived white shoes "
   
 
    return await Runner.run(CustomerComplaint, query, context=query)

import asyncio

async def main():
    result = await run_customer_complaint()
    return result
if __name__ =='__main__':
    result = asyncio.run(main())
    print(result.final_output)

comp_description:we orderd black shoes but we recived white shoes
final_resp:valid=False resp_natural='Thank you for reaching out to us. We are unable to process your complaint at this time because your order is still pending and has not yet been delivered. The expected delivery date is 2025-08-03. Please contact us again after you have received your order.'
{'orders': [{'shipping_address': 'village lakra sialkot,pakistan', 'order_id': 'ASM91411184', 'status': 'pending', 'expected_delivery_date': '2025-08-03', 'created_at': '2025-04-26', 'items': [{'product_name': 'Nike Burrow SE', 'price': 45.97, 'quantity': 2, 'color': 'Black/Black', 'size': 7, 'image': 'https://static.nike.com/a/images/t_default/43929bf5-d412-4267-84fa-a544a9bfe004/W+NIKE+BURROW+SE+NA.png'}]}]}


In [46]:
from agents import Agent, function_tool, AgentHooks
import json
import requests
from datetime import date
from pydantic import BaseModel

class OutputType(BaseModel):
    valid:bool
    resp_natural:str

def dynamic_comp_validator(context, agent):
    user_query = context.context["user_query"]
    order_data = context.context["order_data"]
    today = date.today().strftime("%Y-%m-%d")

    return (
    f"You are a complaint validation assistant for **ArmanShoe Mart**.\n\n"
    f"Today's date is {today}.\n\n"
    f"Your task is to determine whether a customer's complaint is valid based only on the order details and their complaint description.\n\n"
    
    "**Order Details:**\n"
    f"{json.dumps(order_data, indent=2)}\n\n"

    "**Customer Complaint Description:**\n"
    f"{user_query}\n\n"

    "**Validation Rules:**\n"
    "1. The complaint is invalid if **any** of the following is true:\n"
    "   - Today's date is **before** the expected delivery date.\n"
    "   - The order `status` is not `'received'` (e.g., it's still `'pending'`).\n"
    "   → In these cases, explain that the item has not yet been delivered, so no complaint can be accepted.\n\n"

    "2. If the complaint mentions incorrect size, color, or quantity, strictly verify that these claims match the order data.\n"
    "   - If the customer says they ordered size 10 but the order has size 7 or 9 → it's invalid.\n"
    "   - If the customer says they ordered 10 items, but the order data shows they only ordered 2 — not received 2, but ordered 2 — then the complaint is invalid.\n"
    "   - Only declare it valid if **all the issues mentioned** clearly exist in the order data.\n"
    "3. Do not make assumptions, and never be lenient. Follow the facts.\n\n"

    "**Your Response Format:**\n"
    "```json\n"
    "{\n"
    '  "valid": true/false,\n'
    '  "resp_natural": "Your polite response to the customer in clear natural language."\n'
    "}\n"
    "```\n\n"
    "Use friendly and professional language in the `resp_natural` field to explain whether the complaint is valid or not. Make sure it's helpful and concise."
    "**Examples:**\n"
    "- User says they ordered 10 shoes, but system shows 4 → invalid.\n"
    "- User says they received wrong color, but order color matches → invalid.\n"
    "- Complaint before delivery date → invalid.\n"
    "- Everything matches complaint → valid.\n\n"

    "**Important:** Always validate strictly, based on order facts. Never guess. Never assume. Only confirm if complaint matches data exactly."
    "**Important:** Compare against what was actually ordered, not what the user says they received. Do not assume under-delivery unless the data supports it"
)


class CustomHook(AgentHooks):

    async def on_tool_end(self, context, agent, tool, output):
        try:
            new_context = {"user_query": context.context, "order_data": output} 
            comp_validator = Agent(
                name="complaint_validator",
                instructions=dynamic_comp_validator,model=model,output_type=OutputType
              
            )
            resp = await Runner.run(comp_validator, input="",  context=new_context,)
            final_resp = resp.final_output
            print(f'final_resp:{final_resp}')
            return final_resp

        except Exception as e:
            err = str(e)
            if "API key not valid" in err or "API_KEY_INVALID" in err:
                code, msg = "400", "Invalid API key. Please provide a valid API key."
            elif "RESOURCE_EXHAUSTED" in err or "quota" in err.lower():
                code, msg = (
                    "429",
                    "Quota exceeded. Please upgrade or wait before retrying.",
                )
            elif isinstance(e, requests.exceptions.ConnectionError):
                code, msg = "500", "Network error. Please check your connection."
            elif "model is overloaded" in err.lower() or "503" in err:
                code, msg = (
                    "503",
                    "Gemini model is overloaded. Please try again after a few seconds.",
                )
            else:
                code, msg = "API_ERROR", f"Unexpected API error: {err}"

            raise RuntimeError(f"{code}: {msg}") from e


@function_tool
def customer_complaint_fun(email: str, tracking_id: str, comp_description: str) -> dict:
    """
    Register and return complaint-related details for a user's order.

    Args:
        email (str): The customer's email address.
        tracking_id (str): The order tracking ID the complaint relates to.
        comp_description (str): A description of the issue or complaint.

    Returns:
        dict: Complaint summary, order details, and item-level breakdown.
    """
    print(f"comp_description:{comp_description}")
    resp = {
        "orders": [
            {
                "shipping_address": "village lakra sialkot,pakistan",
                "order_id": "ASM91411184",
                "status": "pending",
                "expected_delivery_date": "2025-08-03",
                "created_at": "2025-04-26",
                "items": [
                    {
                        "product_name": "Nike Burrow SE",
                        "price": 45.97,
                        "quantity": 2,
                        "color": "Black/Black",
                        "size": 7,
                        "image": "https://static.nike.com/a/images/t_default/43929bf5-d412-4267-84fa-a544a9bfe004/W+NIKE+BURROW+SE+NA.png",
                    },
                   
                ],
            }
        ]
    }
    return resp


CustomerComplaint = Agent(
    name="customer_complaint_agent",
    instructions="""You are a customer support assistant helping users submit complaints related to their orders.\n"
        "Use the customer's email, order tracking ID, and complaint description to register the complaint "
        "and retrieve related order information.\n\n"
        "If any required fields are missing, politely ask the user to provide them.\n"
        "You must call the appropriate tool with complete data.""",
    model=model,
    tools=[customer_complaint_fun],
    tool_use_behavior="stop_on_first_tool",
    hooks=CustomHook(),
)


async def run_customer_complaint():
    query =  "i want to submit my complaint my email is armanashraf015@gmail.com and my tracking id is ASM91411184 we orderd black shoes but we recived white shoes "
   
 
    return await Runner.run(CustomerComplaint, query, context=query)

import asyncio

async def main():
    result = await run_customer_complaint()
    return result
if __name__ =='__main__':
    result = asyncio.run(main())
    print(result.final_output)

comp_description:we orderd black shoes but we recived white shoes
final_resp:valid=False resp_natural='Thank you for reaching out to ArmanShoe Mart. I have reviewed your order with tracking ID ASM91411184. I see that the order is still pending and the expected delivery date is 2025-08-03. As the order has not yet been delivered, we cannot accept a complaint at this time. Please contact us again after you have received your order.'
{'orders': [{'shipping_address': 'village lakra sialkot,pakistan', 'order_id': 'ASM91411184', 'status': 'pending', 'expected_delivery_date': '2025-08-03', 'created_at': '2025-04-26', 'items': [{'product_name': 'Nike Burrow SE', 'price': 45.97, 'quantity': 2, 'color': 'Black/Black', 'size': 7, 'image': 'https://static.nike.com/a/images/t_default/43929bf5-d412-4267-84fa-a544a9bfe004/W+NIKE+BURROW+SE+NA.png'}]}]}


In [28]:
from agents import function_tool
from typing import Dict

# Simulated database
fake_order_db = {
    "SKDE34343": {
        "status": "In Transit",
        "estimated_delivery": "2025-07-25",
        "location": "Lahore Hub"
    },
    "XYZ123456": {
        "status": "Delivered",
        "estimated_delivery": "2025-07-19",
        "location": "Karachi"
    }
}

@function_tool
def lookup_order_status(tracking_id: str) -> Dict[str, str]:
    """Lookup order status by tracking ID."""
    order = fake_order_db.get(tracking_id.upper())
    if not order:
        print(resp)
        resp = {
            "status": "Not Found",
            "estimated_delivery": "N/A",
            "location": "No record found for this tracking ID."
        }
    print(order)    
    return order
from agents import Agent
from pydantic import BaseModel

class OrderTrackingInput(BaseModel):
    tracking_id: str

class OrderTrackingOutput(BaseModel):
    status: str
    estimated_delivery: str
    location: str

order_tracking = Agent(
    name="order_tracking",
    instructions=(
        "You are an order tracking assistant. "
        "Extract the tracking ID from the user’s message and use the `lookup_order_status` tool to get real order info.\n\n"
        "Only respond using the result of the tool call — do not generate your own status.\n\n"
        "If the tracking ID is not found in the user message, raise an error."
    ),
    model=model,
    # input_type=OrderTrackingInput,
    output_type=OrderTrackingOutput,
    # tool_use_behavior='stop_on_first_tool',
    tools=[lookup_order_status],  # ✅ Connect to backend tool
)
from agents import Runner
import asyncio

async def main():
    query = "My order ID . Can you tell me the status?"
    result = await Runner.run(order_tracking, query)
    print("Status:", result.final_output) 

if __name__ == "__main__":
    asyncio.run(main())


Status: status='string' estimated_delivery='string' location='string'


In [ ]:
async def main(query: str):  
    selected_agent = ['ProductRecommendation', 'CustomerComplaint']
    print(selected_agent)
    if selected_agent is None:
        raise ValueError(f"Unrecognized intent '{selected_agent}'. Please make sure the categorizer returns a valid label.")
    result = await Runner.run(selected_agent, query) 
    return result.final_output
if __name__ =="__main__":
    asyncio.run(main('my name is arman and i want to buy red shoes'))     

['ProductRecommendation', 'CustomerComplaint']


In [2]:
from agents import Agent, HandoffInputFilter, handoff
from agents.exceptions import handoff_filters
refund_agent = Agent(name="Refund agent",model=model)


triage_agent = Agent(name="Triage agent", handoffs=[handoff(refund_agent,input_filter=handoff_filters.remove_all_tools,)],model=model)


ImportError: cannot import name 'handoff_filters' from 'agents.exceptions' (c:\Users\arman\AppData\Local\Programs\Python\Python313\Lib\site-packages\agents\exceptions.py)